# Example

This notebook illustrate how to calculate RCA in parallel using this repository.

In [1]:
from src.parallel_calc import *
from src.rca_calc import *
from constants import *
from functools import partial

Prepare: Create an instance of `Parallel_Calculator` class.

In [2]:
parallel = Parallel_Calculator()

1. Calculate "xij" values for all files

In [3]:
xij_process_file = partial(parallel.run_xij, prod=PROD)
parallel.parallel_run(FOLDER_PATH, xij_process_file, "xij")

Processing BACI_HS12_Y2014_V202501.csv in thread: 123145684119552
Processing BACI_HS12_Y2012_V202501.csv in thread: 123145700909056
Processing BACI_HS12_Y2013_V202501.csv in thread: 123145717698560
BACI_HS12_Y2014_V202501.csv is done.
BACI_HS12_Y2013_V202501.csv is done.
BACI_HS12_Y2012_V202501.csv is done.
Total execution time: 0.04 seconds


2. Calculate "xin" values for all files

In [4]:
xin_process_file = partial(parallel.run_xin, val=VAL, prod=PROD)
parallel.parallel_run(FOLDER_PATH, xin_process_file, "xin", XIN_NAMES)

Processing BACI_HS12_Y2014_V202501.csv in thread: 123145684119552Processing BACI_HS12_Y2012_V202501.csv in thread: 123145700909056

Processing BACI_HS12_Y2013_V202501.csv in thread: 123145717698560
BACI_HS12_Y2012_V202501.csv is done.
BACI_HS12_Y2013_V202501.csv is done.
BACI_HS12_Y2014_V202501.csv is done.
Total execution time: 1.04 seconds


3. Calculate "xwj" values for all files

In [5]:
xwj_process_file = partial(parallel.run_xwj, val=VAL)
parallel.parallel_run(FOLDER_PATH, xwj_process_file, "xwj", XWJ_NAMES)

Processing BACI_HS12_Y2014_V202501.csv in thread: 123145684119552
Processing BACI_HS12_Y2012_V202501.csv in thread: 123145700909056
Processing BACI_HS12_Y2013_V202501.csv in thread: 123145717698560
Handling 32 exporter.
Handling 32 exporter.
Handling 4 exporter.
Handling 36 exporter.Handling 36 exporter.

Handling 8 exporter.
Handling 32 exporter.Handling 40 exporter.

Handling 40 exporter.
Handling 50 exporter.
Handling 36 exporter.
Handling 50 exporter.
Handling 56 exporter.
Handling 40 exporter.
Handling 56 exporter.
Handling 50 exporter.
Handling 72 exporter.
Handling 60 exporter.
Handling 56 exporter.
Handling 76 exporter.
Handling 68 exporter.
Handling 60 exporter.
Handling 90 exporter.
Handling 70 exporter.
Handling 76 exporter.
Handling 100 exporter.
Handling 72 exporter.
Handling 90 exporter.
Handling 100 exporter.
Handling 104 exporter.
Handling 76 exporter.
Handling 104 exporter.
Handling 112 exporter.
Handling 84 exporter.
Handling 108 exporter.
Handling 116 exporter.
Handl

4. Calculate "xwn" values for all files

In [6]:
xwn_process_file = partial(parallel.run_xwn, val=VAL)
parallel.parallel_run(FOLDER_PATH, xwn_process_file, "xwn", XWN_NAMES)

Processing BACI_HS12_Y2014_V202501.csv in thread: 123145684119552Processing BACI_HS12_Y2012_V202501.csv in thread: 123145700909056

Processing BACI_HS12_Y2013_V202501.csv in thread: 123145717698560
BACI_HS12_Y2013_V202501.csv is done.
BACI_HS12_Y2014_V202501.csv is done.
BACI_HS12_Y2012_V202501.csv is done.
Total execution time: 0.38 seconds


5. Calculate RCA values for all files

In [ ]:
file_path_list = [f"{os.getcwd()}/xij.csv", 
                      f"{os.getcwd()}/xin.csv", 
                      f"{os.getcwd()}/xwj.csv", 
                      f"{os.getcwd()}/xwn.csv"]
    
max_workers = os.cpu_count() * 2 if os.cpu_count() else 4 
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(parallel.run_rca, val, file_path_list): val for val in VAL}
    
    dfs = [pd.read_csv(f"{os.getcwd()}/xij.csv", dtype={'Year': int, 'Importer': int, 'Exporter': int}).iloc[:, :4]]
    for future in as_completed(futures):
        df = future.result()
        dfs.append(df)

final_df = pd.concat(dfs, axis=1)
final_df.to_csv("rca.csv", index=False)

Processing in thread: 123145684119552
Processing in thread: 123145700909056


FileNotFoundError: [Errno 2] No such file or directory: '/Users/bp45th/Documents/github/Revealed-Comparative-Advantage-Calculator/results_new/xij.csv'